# Import Libraries and set seeds

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pylab
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

import json
import pickle
import random as python_random
import os


import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50, InceptionResNetV2

from keras import models, layers
from keras.optimizers import RMSprop, SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint

np.random.seed(123)
python_random.seed(123)
tf.random.set_seed(1234)

In [ ]:
# Check for results folder (create if doesn't exist)
base_folder = './results'
if not os.path.exists('./results'):
    os.mkdir('./results')

subfolders = ['mc', 'histories', 'acc_loss_plots', 'roc_auc', 'cm', 'cls_rpt']

for sf in subfolders:
    filepath = '{}/{}'.format(base_folder, subfolders)
    if not os.path.exists(filepath):
        os.mkdir(filepath)

# Load Dataset

In [ ]:
df = pd.read_csv('./meta.csv')

train_df, test_df = train_test_split(df, test_size=0.2, random_state=0, stratify=df['gender'])
train_df, val_df = train_test_split(train_df, test_size=0.3, random_state=0, stratify=train_df['gender'])

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Functions

In [ ]:
def generate_data(train_df, val_df, test_df, img_size, batch_size):
  # data generator for training data
  train_generator = train_datagen.flow_from_dataframe(train_df, x_col="path", 
                                                      y_col="gender", 
                                                      target_size=(img_size,img_size), 
                                                      batch_size=batch_size, 
                                                      class_mode='binary')

  # data generator for validation data
  validation_generator = validation_datagen.flow_from_dataframe(val_df, x_col="path", 
                                                                y_col="gender", 
                                                                target_size=(img_size,img_size),
                                                                batch_size=batch_size, 
                                                                class_mode='binary')

  # data generator for validation data to be used for prediction
  predict_generator = validation_datagen.flow_from_dataframe(val_df, x_col="path", 
                                                             y_col="gender", 
                                                             target_size=(img_size,img_size),
                                                             batch_size=batch_size, 
                                                             class_mode='binary',
                                                             shuffle = False)

  # data generator for testing data
  test_generator = test_datagen.flow_from_dataframe(test_df, x_col="path", 
                                                    y_col="gender", 
                                                    target_size=(img_size,img_size),
                                                    batch_size=batch_size, 
                                                    class_mode='binary',
                                                    shuffle = False)
  
  generators = {'train_gen': train_generator,
                'validation_gen': validation_generator,
                'test_gen': test_generator,
                'predict_gen': predict_generator}
  
  return generators

In [ ]:
# Define Model Checkpoint callback
def mc(title, save_path):
    file_path = '{}/mc/{}_mc.h5'.format(save_path, title)
    return ModelCheckpoint(file_path, monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
# Define EarlyStopping callback
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7, restore_best_weights=True)

In [ ]:
def save_history(history, save_path, title):
    # Saving history to file
    filename = '{}/histories/{}_hist.pickle'.format(save_path, title)
    with open(filename, 'wb') as file_pi:
        pickle.dump(history, file_pi)
    print("Successfully saved history file at {}".format(filename))

In [ ]:
def fit(model, epochs, title, generator, es, save_path):
    train_gen = generator['train_gen']
    validation_gen = generator['validation_gen']
    history = model.fit(train_gen, 
                      steps_per_epoch=train_gen.samples/train_gen.batch_size, 
                      epochs=epochs,
                      validation_data=validation_gen,
                      validation_steps=validation_gen.samples/validation_gen.batch_size,
                      verbose=2,
                      callbacks=[es, mc(title, save_path)])

    save_history(history.history, save_path, title)

    return history.history

In [ ]:
def model_analysis(model, generator, title, save_path):
    print('Results for {}'.format(title))
    test_gen = generator['predict_gen']
    prediction = model.predict(test_gen,
                             steps=test_gen.samples/test_gen.batch_size,
                             verbose=2)
    predicted_classes = prediction.flatten()
    predicted_classes[predicted_classes>=0.5] = 1
    predicted_classes[predicted_classes<0.5] = 0

    actual = test_gen.classes
    errors = np.where(predicted_classes != actual)[0]
    print("Error rate {}".format(len(errors)/test_gen.samples))

    genders = ['Female', 'Male']
    cm = confusion_matrix(actual, predicted_classes)
    sns.heatmap(cm, annot=True, cmap='Blues', fmt="d",
              xticklabels=genders, yticklabels=genders)

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig('{}/cm/{}_cm.pdf'.format(save_path, title))
    plt.close()

    print(classification_report(actual, predicted_classes, target_names=genders))
    clsf_report = pd.DataFrame(classification_report(actual, predicted_classes, target_names=genders, output_dict=True)).transpose()
    clsf_report.to_csv('{}/cls_rpt/{}_cr.csv'.format(save_path, title), index= True)

    # Get AUC ROC Score and save as a file
    roc_auc = roc_auc_score(actual, predicted_classes)
    roc_auc_file = '{}/roc_auc/{}_roc_auc'.format(save_path, title)
    params = {'title': title,
            'roc_auc': roc_auc}
    with open(roc_auc_file, 'w') as fp:
    json.dump(params, fp)
    print("Successfully saved auc_roc file at {}".format(roc_auc_file))

In [ ]:
def base_model_plot(history, title, save_path=None):
    pylab.figure(figsize=(12,4))
    pylab.subplot(1,2,1)
    if 'acc' in history.keys():
        pylab.plot(history['acc'], label='train')
        pylab.plot(history['val_acc'], label='validation')
    else:
        pylab.plot(history['accuracy'], label='train')
        pylab.plot(history['val_accuracy'], label='validation')
    pylab.title('Model Accuracy for {}'.format(title))
    pylab.xlabel('epochs')
    pylab.ylabel('accuracy')
    pylab.legend(loc='best')

    pylab.subplot(1,2,2)
    pylab.plot(history['loss'], label='train')
    pylab.plot(history['val_loss'], label='validation')
    pylab.title('Model Loss for {}'.format(title))
    pylab.xlabel('epochs')
    pylab.ylabel('loss')
    pylab.legend(loc='best')

    if save_path:
        pylab.savefig('{}/acc_loss_plots/{}.pdf'.format(save_path, title))

In [ ]:
def param_opt_plot(hist_dict, title, save_path=None):
    pylab.figure(figsize=(12,4))
    pylab.subplot(1,2,1)
    for key, hist in hist_dict.items():
        if 'acc' in hist_dict[title].keys():
            pylab.plot(hist['val_acc'], label=key)
        else:
            # key is in format of e.g. inceptionresnetv2_bs64 so split key and take index 1 value
            pylab.plot(hist['val_accuracy'], label=key.split('_')[1])
    # title is in the format of e.g. inceptionresnetv2_bs
    pylab.title('Validation Accuracy for {}'.format(title))
    pylab.xlabel('epochs')
    pylab.ylabel('accuracy')
    pylab.legend(loc='best')

    pylab.subplot(1,2,2)
    for key, hist in hist_dict.items():
        pylab.plot(hist['val_loss'], label=key.split('_')[1])
    pylab.title('Validation Loss for {}'.format(title))
    pylab.xlabel('epochs')
    pylab.ylabel('loss')
    pylab.legend(loc='best')

    if save_path:
    pylab.savefig('{}/acc_loss_plots/{}.pdf'.format(save_path, title))

# Train with base model set-up

In [ ]:
# Base Model Parameters
IMG_SIZE_1 = 224
IMG_SIZE_2 = 299
BATCH_SIZE = 128
EPOCHS = 30
LEARNING_RATE = 1e-4
NUM_NEURONS = 1024
DROPOUT_RATE = 0.5
MOMENTUM = 0.9
OPTIMISER = 'SGD'
SAVE_PATH = base_folder

In [ ]:
# Prepare data for base models
generators_224 = generate_data(train_df, val_df, test_df, IMG_SIZE_1, BATCH_SIZE)
generators_299 = generate_data(train_df, val_df, test_df, IMG_SIZE_2, BATCH_SIZE)

## ResNet50 Base Model

### Set Parameters for Base ResNet50

In [ ]:
# Parameters for Base ResNet50
TITLE = 'resnet50_base'
START_LAYER = 143 # Layer to start unfreezing from

### Create Model for Base ResNet50

In [ ]:
# Download ResNet50 weights
resnet50_conv = ResNet50(weights='imagenet', include_top=False, input_tensor=None, input_shape=(224,224,3))

# freeze all layers except stage 4
for layer in resnet50_conv.layers[:START_LAYER]:
    layer.trainable = False
for layer in resnet50_conv.layers[START_LAYER:]:
    layer.trainable = True

# Build base model
resnet50_base = models.Sequential()
resnet50_base.add(resnet50_conv)    
resnet50_base.add(layers.Flatten())   
resnet50_base.add(layers.Dense(NUM_NEURONS, activation='relu'))
resnet50_base.add(layers.Dropout(DROPOUT_RATE))
resnet50_base.add(layers.Dense(1, activation='sigmoid'))

# Compile base model
resnet50_base.compile(loss='binary_crossentropy', 
                      optimizer=SGD(lr=LEARNING_RATE, momentum=MOMENTUM),
                      metrics=['acc'])

### Train (Fit) Base ResNet50 & save information

In [ ]:
# Train and save the history of training + params of model
resnet50_base_hist = fit(resnet50_base, EPOCHS, TITLE, generators_224, es, SAVE_PATH)

# Do Model Analysis + Prediction
model_analysis(resnet50_base, generators_224, TITLE, SAVE_PATH)

### Plot Histories

In [ ]:
# Plot simple accuracy/loss graphs
base_model_plot(resnet50_base_hist, TITLE, SAVE_PATH)

## InceptionResNet Base Model

### Set Parameters for Base InceptionResNet

In [ ]:
# Parameters for Base InceptionResNet
TITLE = 'inceptionresnetv2_base'
START_LAYER = 759
LAST_LAYER = 780

### Create Model for Base InceptionResNet

In [ ]:
# Download InceptionResNet weights
inceptionresnetv2_conv = InceptionResNetV2(weights='imagenet', include_top=False, input_tensor=None, input_shape=(299,299,3))

# freeze all layers except last block
for layer in inceptionresnetv2_conv.layers[:START_LAYER]:
    layer.trainable = False
for layer in inceptionresnetv2_conv.layers[START_LAYER:]:
    layer.trainable = True

# Build base model
inceptionresnetv2_base = models.Sequential()
inceptionresnetv2_base.add(inceptionresnetv2_conv)             
inceptionresnetv2_base.add(layers.Flatten())          
inceptionresnetv2_base.add(layers.Dense(NUM_NEURONS, activation='relu'))
inceptionresnetv2_base.add(layers.Dropout(DROPOUT_RATE))
inceptionresnetv2_base.add(layers.Dense(1, activation='sigmoid'))

# Compile base model
inceptionresnetv2_base.compile(loss='binary_crossentropy', 
                     optimizer=SGD(lr=LEARNING_RATE, momentum=MOMENTUM),
                     metrics=['acc'])

### Train (Fit) Base InceptionResNet & save information

In [ ]:
# Train and save the history of training + params of model
inceptionresnetv2_base_hist = fit(inceptionresnetv2_base, EPOCHS, TITLE, generators_299, es, SAVE_PATH)

In [ ]:
# Plot simple accuracy/loss graphs
base_model_plot(inceptionresnetv2_base_hist, TITLE, SAVE_PATH)

# Determine Better Model between ResNet50 and InceptionResnetV2

In [ ]:
inceptionresnetv2_max = np.max(inceptionresnetv2_base_hist['val_acc'])
resnet50_max = np.max(resnet50_base_hist['val_acc'])

if resnet50_max > inceptionresnetv2_max:
    better_mod = 'resnet50'
else:
    better_mod = 'inceptionresnetv2'

print('resnet50 max val acc: {}'.format(resnet50_max))
print('inceptionresnetv2 max val acc: {}'.format(resnet50_max))
print('The better model is {}'.format(better_mod))

# InceptionResNet Tuning

## Batch Size
### Set Parameters for Batch Sizes

In [ ]:
TITLE = 'inceptionresnetv2_bs'
BATCH_SIZE = [64, 256]
EPOCHS = 30
LEARNING_RATE = 1e-4
NUM_NEURONS = 1024
DROPOUT_RATE = 0.5
MOMENTUM = 0.9
OPTIMISER = 'SGD'
START_LAYER = 759
LAST_LAYER = 780

bs_generators = {}
# Prepare data for base model
for bs in BATCH_SIZE:
    bs_generators[bs] = generate_data(train_df, val_df, test_df, IMG_SIZE_2, bs)

### Create Model for Batch Sizes

In [ ]:
inceptionresnetv2_bs_models = []
for bs in BATCH_SIZE:
    # Download InceptionResNet weights
    inceptionresnetv2_conv = InceptionResNetV2(weights='imagenet', include_top=False, 
                                      input_tensor=None, input_shape=(299,299,3))
    
    # freeze all layers except last block
    for layer in inceptionresnetv2_conv.layers[:START_LAYER]:
        layer.trainable = False
    for layer in inceptionresnetv2_conv.layers[START_LAYER:]:
        layer.trainable = True
    
    # Build model
    model = models.Sequential()
    model.add(inceptionresnetv2_conv)            
    model.add(layers.Flatten())          
    model.add(layers.Dense(NUM_NEURONS, activation='relu'))
    model.add(layers.Dropout(DROPOUT_RATE))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # append model to list
    inceptionresnetv2_bs_models.append(model)

### Train (Fit) Batch Sizes Model & save information

In [ ]:
# Train and save the history of training + params of model
bs_history = {}

for model, bs, gen in zip(inceptionresnetv2_bs_models, BATCH_SIZE, bs_generators):
    model.compile(loss='binary_crossentropy', 
                  optimizer=SGD(lr=LEARNING_RATE, momentum=MOMENTUM),
                  metrics=['acc'])
    
    title = '{}{}'.format(TITLE, bs)
    print('Running {}'.format(title))
    bs_history[title] = fit(model, EPOCHS, title, bs_generators[gen], es, SAVE_PATH)

In [ ]:
# Plot simple accuracy/loss graphs
for title, history in bs_history.items():
    base_model_plot(history, title, SAVE_PATH)
    
# Add base model (bs128) into history dict
key = '{}{}'.format(TITLE, 128)
bs_history[key] = inceptionresnetv2_base_hist
    
# Plot combined val acc/ val loss graphs
param_opt_plot(bs_history, TITLE, SAVE_PATH)

In [ ]:
# Optimal Batch Size
max_acc = 0
best = None
for name, hist in bs_history:
    max_val_acc = np.max(hist['val_acc'])
    if max_val_acc > max_acc:
        max_acc = max_val_acc
        best = name

print('The optimal batch size is {} with max validation accuracy of {}'.format(best, max_acc))

## Dropout

### Set Parameters for Dropouts

In [ ]:
TITLE = 'inceptionresnetv2_dropout'
BATCH_SIZE = 128
EPOCHS = 30
LEARNING_RATE = 1e-4
NUM_NEURONS = 1024
DROPOUT_RATE = [0, 0.2]
MOMENTUM = 0.9
OPTIMISER = 'SGD'
START_LAYER = 759
LAST_LAYER = 780

### Create Model for Dropouts

In [ ]:
inceptionresnetv2_dropout_models = []
for dropout in DROPOUT_RATE:
    # Download InceptionResNet weights
    inceptionresnetv2_conv = InceptionResNetV2(weights='imagenet', include_top=False, 
                                      input_tensor=None, input_shape=(299,299,3))
    
    # freeze all layers except last block
    for layer in inceptionresnetv2_conv.layers[:START_LAYER]:
        layer.trainable = False
    for layer in inceptionresnetv2_conv.layers[START_LAYER:]:
        layer.trainable = True
    
    # Build model
    model = models.Sequential()
    model.add(inceptionresnetv2_conv)
    model.add(layers.Flatten())
    model.add(layers.Dense(NUM_NEURONS, activation='relu'))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # append model to list
    inceptionresnetv2_dropout_models.append(model)

### Train (Fit) Dropout Model & save information

In [ ]:
# Train and save the history of training + params of model
dropout_history = {}

for model, dropout in zip(inceptionresnetv2_dropout_models, DROPOUT_RATE):
    model.compile(loss='binary_crossentropy', 
                  optimizer=SGD(lr=LEARNING_RATE, momentum=MOMENTUM),
                  metrics=['acc'])
    
    title = '{}{}'.format(TITLE, dropout)
    print('Running {}'.format(title))
    dropout_history[dropout] = fit(model, EPOCHS, title, generators_299, es, SAVE_PATH)

In [ ]:
# Plot simple accuracy/loss graphs
for title, history in dropout_history.items():
    base_model_plot(history, title, SAVE_PATH)
    
# Add base model (bs128) into history dict
key = '{}{}'.format(TITLE, 0.5)
dropout_history[key] = inceptionresnetv2_base_hist
    
# Plot combined val acc/ val loss graphs
param_opt_plot(dropout_history, TITLE, SAVE_PATH)

In [ ]:
# Optimal Dropout Rate
max_acc = 0
best = None
for name, hist in dropout_history:
    max_val_acc = np.max(hist['val_acc'])
    if max_val_acc > max_acc:
        max_acc = max_val_acc
        best = name

print('The optimal dropout rate is {} with max validation accuracy of {}'.format(best, max_acc))

## Learning Rate

### Set Parameters for Learning Rate

In [ ]:
TITLE = 'inceptionresnetv2_lr'
BATCH_SIZE = 128
EPOCHS = 30
LEARNING_RATE = [1e-2, 1e-3]
NUM_NEURONS = 1024
DROPOUT_RATE = 0.5
MOMENTUM = 0.9
OPTIMISER = 'SGD'
START_LAYER = 759
LAST_LAYER = 780

### Create Model for Dropouts

In [ ]:
inceptionresnetv2_lr_models = []
for lr in LEARNING_RATE:
    # Download InceptionResNet weights
    inceptionresnetv2_conv = InceptionResNetV2(weights='imagenet', include_top=False, input_tensor=None, input_shape=(299,299,3))
    
    # freeze all layers except last block
    for layer in inceptionresnetv2_conv.layers[:START_LAYER]:
        layer.trainable = False
    for layer in inceptionresnetv2_conv.layers[START_LAYER:]:
        layer.trainable = True
        
    # Build model
    model = models.Sequential()
    model.add(inceptionresnetv2_conv)
    model.add(layers.Flatten())
    model.add(layers.Dense(NUM_NEURONS, activation='relu'))
    model.add(layers.Dropout(DROPOUT_RATE))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # append model to list
    inceptionresnetv2_lr_models.append(model)

### Train (Fit) Dropout Model & save information

In [ ]:
# Train and save the history of training + params of model
lr_history = {}

for model, lr in zip(inceptionresnetv2_lr_models, LEARNING_RATE):
    model.compile(loss='binary_crossentropy', 
                  optimizer=SGD(lr=lr, momentum=MOMENTUM),
                  metrics=['acc'])
    
    title = '{}{}'.format(TITLE, lr)
    print('Running {}'.format(title))
    lr_history[lr] = fit(model, EPOCHS, title, generators_299, es, SAVE_PATH)

In [ ]:
# Plot simple accuracy/loss graphs
for title, history in lr_history.items():
    base_model_plot(history, title, SAVE_PATH)
    
# Add base model (bs128) into history dict
key = '{}{}'.format(TITLE, 1e-4)
lr_history[key] = inceptionresnetv2_base_hist
    
# Plot combined val acc/ val loss graphs
param_opt_plot(lr_history, TITLE, SAVE_PATH)

In [ ]:
# Optimal Learning Rate
max_acc = 0
best = None
for name, hist in lr_history:
    max_val_acc = np.max(hist['val_acc'])
    if max_val_acc > max_acc:
        max_acc = max_val_acc
        best = name

print('The optimal learning rate is {} with max validation accuracy of {}'.format(best, max_acc))

## Stacked

### Set Parameters for Stacked

In [ ]:
TITLE = 'inceptionresnetv2_stacked'
BATCH_SIZE = 128
EPOCHS = 30
LEARNING_RATE = 1e-4
NUM_NEURONS = 1024
DROPOUT_RATE = 0.5
MOMENTUM = 0.9
OPTIMISER = 'SGD'
START_LAYER = 759
LAST_LAYER = 780

### Create Model for Stacked

In [ ]:
# Download InceptionResNet weights
inceptionresnetv2_conv = InceptionResNetV2(weights='imagenet', include_top=False, input_tensor=None, input_shape=(299,299,3))

# freeze all layers except last block
for layer in inceptionresnetv2_conv.layers[:START_LAYER]:
    layer.trainable = False
for layer in inceptionresnetv2_conv.layers[START_LAYER:]:
    layer.trainable = True

# Build model
stacked_model = models.Sequential()
stacked_model.add(inceptionresnetv2_conv)
stacked_model.add(layers.Flatten())
stacked_model.add(layers.Dense(NUM_NEURONS, activation='relu'))
stacked_model.add(layers.Dropout(DROPOUT_RATE))
stacked_model.add(layers.Dense(NUM_NEURONS, activation='relu'))
stacked_model.add(layers.Dropout(DROPOUT_RATE))
stacked_model.add(layers.Dense(1, activation='sigmoid'))

### Train (Fit) Stacked Model & save information

In [ ]:
# Train and save the history of training + params of model
stacked_model.compile(loss='binary_crossentropy', 
                      optimizer=SGD(lr=LEARNING_RATE, momentum=MOMENTUM),
                      metrics=['acc'])

stacked_history = {}
stacked_history['inceptionresnetv2_stacked'] = fit(stacked_model, EPOCHS, TITLE, generators_299, es, SAVE_PATH)

In [ ]:
# Plot simple accuracy/loss graphs
for title, history in stacked_history.items():
    base_model_plot(history, title, SAVE_PATH)
    
# Add base model (bs128) into history dict
key = 'inceptionresnetv2_unstacked'
stacked_history[key] = inceptionresnetv2_base_hist
    
# Plot combined val acc/ val loss graphs
param_opt_plot(stacked_history, 'inceptionresnetv2_layered', SAVE_PATH)

In [ ]:
# Optimal Number of Layers
max_acc = 0
best = None
for name, hist in stacked_history:
    max_val_acc = np.max(hist['val_acc'])
    if max_val_acc > max_acc:
        max_acc = max_val_acc
        best = name

print('The optimal number of layers is {} with max validation accuracy of {}'.format(best, max_acc))

# Save my optimal model:
Based on results from previous parts, optimal model is:
1. Batch Size = 128
2. Dropout Rate = 0.5
3. Learning Rate = 0.01
4. Number of Fully-Connected Layers = 1

In [ ]:
optimal_history = lr_history[1e-2]
save_history(optimal_history, SAVE_PATH, 'inceptionresnetv2_optimal')

# Plot all 6 base models

In [ ]:
# Put all my plots in this folder and then can run together
directory = './results/histories'

base_histories = {}

for filename in os.listdir(directory):
    if 'base' in filename:
        filepath = os.path.join(directory, filename)
        print('Found file at: {}'.format(filepath))

        with open(filename) as json_file:
            base_histories[filename] = json.load(json_file)

In [ ]:
# Plot combined val acc/ val loss graphs
param_opt_plot(base_histories, 'base_models', SAVE_PATH)

# Plot all optimal models

In [ ]:
# Put all my plots in this folder and then can run together
directory = './results/histories'

optimal_histories = {}

for filename in os.listdir(directory):
    if 'optimal' in filename:
        filepath = os.path.join(directory, filename)
        print('Found file at: {}'.format(filepath))

        with open(filename) as json_file:
            optimal_histories[filename] = json.load(json_file)

In [ ]:
# Plot combined val acc/ val loss graphs
param_opt_plot(optimal_histories, 'optimal_models', SAVE_PATH)

# Plot Transfer Learning Models

[Nithya] I'm not sure how you are going to name them but you can either
1. Name them in a way that there is a word that only models in this part has then read in everything + the optimal model OR
2. Call the param_opt_plot function directly after your finetuning if you have a dictionary containing the 4 models
 (unfreezeAll, unfreeze1Top, unfreeze2Top and optimal)

In [ ]:
# Put all my plots in this folder and then can run together
directory = './results/histories'

tl_histories = {}

for filename in os.listdir(directory):
    if 'tl' in filename or 'vgg16_optimal' in filename:
        filepath = os.path.join(directory, filename)
        print('Found file at: {}'.format(filepath))

        with open(filename) as json_file:
            tl_histories[filename] = json.load(json_file)

In [ ]:
# Plot combined val acc/ val loss graphs
param_opt_plot(tl_histories, 'tl_models', SAVE_PATH)

# Print the maximum validation accuracy for all models trained in project

In [ ]:
# Plot Max Validation Accuracies
all_histories = {}

for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    print('Found file at: {}'.format(filepath))
    with open(filename) as json_file:
        all_histories[filename] = json.load(json_file)

In [ ]:
# Print max val acc results and epoch and save to a dictionary
max_val_acc = {}
overall_max = 0
max_model = None
print('Max Validation Accuracy for each model')
for model, hist in all_histories.items():
    if 'acc' in hist.keys():
        max_val_acc[model] = np.max(hist['val_acc'])
        epoch = np.argmax(history['val_acc']) + 1
    else:
        max_val_acc[model] = np.max(hist['val_accuracy'])
        epoch = np.argmax(hist['val_accuracy']) + 1
    
    print('{:<25}: {:.4f}'.format(model, max_val_acc[model]))
    if max_val_acc[model] > overall_max:
        overall_max = max_val_acc[model]
        max_model = model
print()
print('The best model is {} with {:.4f}'.format(max_model, overall_max))

In [ ]:
# convert to dataframe and save as csv file
df = pd.DataFrame(list(max_val_acc.items()),columns = ['model', 'max_val_acc']) 
df_sorted = df.sort_values(by=['max_val_acc'])
df_sorted.to_csv('./results/all_max_val_acc.csv')